In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
# Define the Q-Network model
def create_q_network(input_shape, action_space):
    model = Sequential(
        [
            tf.keras.Input(shape=input_shape),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(action_space, activation='linear')
        ]
    )
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
    return model

In [ ]:
class ENV():
    def __init__(self):
        self.env_col = 11
        self.env_row = 11
        self.state = (0, 0)
        self.action_space = 4
        self.reward = 0
        self.done = False
        self.goal = (3, 3)

    def reset(self):
        self.state = (0, 0)
        self.reward = 0
        self.done = False
        return self.state
    
    def step(self, action):
        if action == 0: # up
            self.state = (self.state[0], self.state[1] + 1)
        elif action == 1: # down
            self.state = (self.state[0], self.state[1] - 1)
        elif action == 2: # left
            self.state = (self.state[0] - 1, self.state[1])
        elif action == 3: # right
            self.state = (self.state[0] + 1, self.state[1])
        else:
            raise ValueError("Invalid action")
        if self.state == self.goal:
            self.reward = 100
            self.done = True
        elif self.state[0] < 0 or self.state[0] >= self.env_col or self.state[1] < 0 or self.state[1] >= self.env_row:
            self.reward = -100
            self.done = True
        else:
            self.reward = -1
        return self.state, self.reward, self.done


In [ ]:
#Define the DQN agent
class DQNAgent:
    def __init__(self, state_shape, action_space):
        self.state_shape = state_shape
        self.action_space = action_space
        self.memory = []
        self.gamma = 0.95  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_decay = 0.995  # Decay rate for exploration rate
        self.epsilon_min = 0.01  # Minimum exploration rate
        self.model = create_q_network(state_shape, action_space)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.action_space)
        print("state in action: ",state)
        state = np.reshape(state, self.state_shape)
        state = tf.convert_to_tensor(state)
        state = tf.expand_dims(state, 0)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        # Debug
        # print(self.memory)
        # batch = np.random.choice(self.memory, batch_size, replace=False)
        batch_ch = np.random.choice(len(self.memory), batch_size, replace=False)
        batch = [self.memory[i] for i in batch_ch]
        for state, action, reward, next_state, done in batch:
            state = np.reshape(state, self.state_shape)
            next_state = np.reshape(next_state, self.state_shape)
            target = reward
            if not done:
                print(next_state.shape)
                next_state = tf.convert_to_tensor(next_state)
                next_state = tf.expand_dims(next_state, 0)
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            print(state.shape)
            state = tf.convert_to_tensor(state)
            state = tf.expand_dims(state, 0)
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [ ]:
# Initialize the environment and agent
state_shape = (2,)  # Example state shape, adjust according to your actual state representation # Only x and y coordinates
action_space = 4  # Example action space size, adjust according to your actual actions # 4 actions: up, down, left, right
agent = DQNAgent(state_shape, action_space)
env = ENV()

In [ ]:
#Training loop
num_episodes = 1000  # Set the number of training episodes
batch_size = 32  # Set the batch size for replay
for episode in range(num_episodes):
    state = env.reset()
    state = np.reshape(state, state_shape)
    done = False
    while not done:
        action = agent.act(state)
        next_state, reward, done = env.step(action)
        next_state = np.reshape(next_state, state_shape)
        agent.remember(state, action, reward, next_state, done)
        state = next_state

        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

In [ ]:
# Use the trained agent to control the agent in the environment
state = env.reset()
state = np.reshape(state, state_shape)
done = False
while not done:
    action = agent.act(state)
    next_state, _, done = env.step(action)
    next_state = np.reshape(next_state, state_shape)
    # Take action with the agent in the environment
    state = next_state
    print(state)